In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128'

In [4]:
pretrained = ''
resume = './log/pggan/128x128/checkpoint.pth.tar'

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 200
start_epoch = 0
train_batch = 256
test_batch = 200
lr = 0.1
schedule = [70, 150]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128x128' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')    
train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(val_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

In [9]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 4.01M


In [10]:
# summary(model, input_size=(3,64,64), device='cuda')

# Loss

In [11]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4, nesterov=True)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [12]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

==> Resuming from checkpoint..


# Train

In [13]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [14]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [15]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [16]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)
    
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    scheduler_warmup.step()

    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [31 | 200] LR: 0.782317
1/502 Data:2.028 | Batch:5.241 | Total:0:00:05 | ETA:0:43:46 | Loss:0.41410720348358154 | top1:80.46875
11/502 Data:0.004 | Batch:0.692 | Total:0:00:10 | ETA:0:07:47 | Loss:0.4151278950951316 | top1:81.28551483154297
21/502 Data:0.001 | Batch:0.507 | Total:0:00:14 | ETA:0:03:19 | Loss:0.4172373243740627 | top1:80.78497314453125
31/502 Data:0.001 | Batch:0.453 | Total:0:00:19 | ETA:0:03:42 | Loss:0.41744823609628984 | top1:80.83416748046875
41/502 Data:0.001 | Batch:0.355 | Total:0:00:23 | ETA:0:03:37 | Loss:0.4185655516822164 | top1:80.69740295410156
51/502 Data:0.001 | Batch:0.483 | Total:0:00:28 | ETA:0:03:33 | Loss:0.41998923993578147 | top1:80.53768920898438
61/502 Data:0.003 | Batch:0.425 | Total:0:00:33 | ETA:0:03:41 | Loss:0.42032849984090837 | top1:80.56480407714844
71/502 Data:0.001 | Batch:0.350 | Total:0:00:37 | ETA:0:03:17 | Loss:0.4182482449941232 | top1:80.69982147216797
81/502 Data:0.002 | Batch:0.411 | Total:0:00:42 | ETA:0:03:08 | Loss:0

/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


161/161 Data:0.000 | Batch:0.221 | Total:0:00:23 | ETA:0:00:00 | Loss:0.1755084312314928 | top1:99.64486694335938

Epoch: [32 | 200] LR: 0.170000
1/502 Data:1.418 | Batch:2.065 | Total:0:00:02 | ETA:0:17:15 | Loss:0.4396880865097046 | top1:78.90625
11/502 Data:0.001 | Batch:0.329 | Total:0:00:06 | ETA:0:05:23 | Loss:0.41881704330444336 | top1:80.71733093261719
21/502 Data:0.000 | Batch:0.385 | Total:0:00:11 | ETA:0:03:55 | Loss:0.4172038918449765 | top1:80.72917175292969
31/502 Data:0.001 | Batch:0.482 | Total:0:00:16 | ETA:0:03:57 | Loss:0.4149245581319255 | top1:80.78376770019531
41/502 Data:0.001 | Batch:0.480 | Total:0:00:22 | ETA:0:04:16 | Loss:0.4143368819864785 | top1:80.72599029541016
51/502 Data:0.005 | Batch:0.603 | Total:0:00:27 | ETA:0:04:17 | Loss:0.4172297406430338 | top1:80.4457778930664
61/502 Data:0.005 | Batch:0.614 | Total:0:00:33 | ETA:0:04:22 | Loss:0.41671867351062963 | top1:80.40470886230469
71/502 Data:0.003 | Batch:0.685 | Total:0:00:39 | ETA:0:03:41 | Loss:0.4

191/502 Data:0.001 | Batch:0.390 | Total:0:01:22 | ETA:0:02:43 | Loss:0.40940825352494004 | top1:81.00048828125
201/502 Data:0.001 | Batch:0.381 | Total:0:01:28 | ETA:0:02:36 | Loss:0.40983161923304123 | top1:80.95848846435547
211/502 Data:0.001 | Batch:0.498 | Total:0:01:32 | ETA:0:02:00 | Loss:0.40943518224485675 | top1:81.00007629394531
221/502 Data:0.001 | Batch:0.461 | Total:0:01:36 | ETA:0:02:06 | Loss:0.40925485569966863 | top1:81.02552795410156
231/502 Data:0.003 | Batch:0.373 | Total:0:01:41 | ETA:0:02:11 | Loss:0.40960206084953243 | top1:81.01325988769531
241/502 Data:0.001 | Batch:0.395 | Total:0:01:46 | ETA:0:02:07 | Loss:0.40968541559836674 | top1:81.00849914550781
251/502 Data:0.005 | Batch:0.654 | Total:0:01:51 | ETA:0:02:04 | Loss:0.4097295941347145 | top1:81.0009994506836
261/502 Data:0.001 | Batch:0.541 | Total:0:01:55 | ETA:0:01:45 | Loss:0.4104087162063496 | top1:80.93121337890625
271/502 Data:0.001 | Batch:0.363 | Total:0:02:00 | ETA:0:01:51 | Loss:0.41084010640633

391/502 Data:0.001 | Batch:0.444 | Total:0:03:02 | ETA:0:00:46 | Loss:0.4118112915617121 | top1:80.92231750488281
401/502 Data:0.001 | Batch:0.366 | Total:0:03:05 | ETA:0:00:36 | Loss:0.41183994201353363 | top1:80.89639282226562
411/502 Data:0.004 | Batch:0.348 | Total:0:03:10 | ETA:0:00:42 | Loss:0.41150404748545366 | top1:80.90974426269531
421/502 Data:0.013 | Batch:0.388 | Total:0:03:15 | ETA:0:00:40 | Loss:0.4114149590852425 | top1:80.91876220703125
431/502 Data:0.001 | Batch:0.658 | Total:0:03:20 | ETA:0:00:39 | Loss:0.4115896182104496 | top1:80.912841796875
441/502 Data:0.001 | Batch:0.444 | Total:0:03:25 | ETA:0:00:30 | Loss:0.4115610009823797 | top1:80.92314910888672
451/502 Data:0.003 | Batch:0.662 | Total:0:03:30 | ETA:0:00:28 | Loss:0.41183835063170965 | top1:80.90181732177734
461/502 Data:0.001 | Batch:0.655 | Total:0:03:35 | ETA:0:00:21 | Loss:0.4117998337926678 | top1:80.89835357666016
471/502 Data:0.001 | Batch:0.321 | Total:0:03:40 | ETA:0:00:18 | Loss:0.411566396870683

...

191/502 Data:0.002 | Batch:0.318 | Total:0:01:05 | ETA:0:01:45 | Loss:0.09752306687348176 | top1:96.2369155883789
201/502 Data:0.002 | Batch:0.357 | Total:0:01:09 | ETA:0:01:42 | Loss:0.09749575437449698 | top1:96.23755645751953
211/502 Data:0.001 | Batch:0.384 | Total:0:01:12 | ETA:0:01:46 | Loss:0.09710267059045945 | top1:96.23445129394531
221/502 Data:0.002 | Batch:0.319 | Total:0:01:16 | ETA:0:01:35 | Loss:0.09689155076019365 | top1:96.2439956665039
231/502 Data:0.001 | Batch:0.353 | Total:0:01:19 | ETA:0:01:31 | Loss:0.09682969611896065 | top1:96.25101470947266
241/502 Data:0.002 | Batch:0.335 | Total:0:01:22 | ETA:0:01:27 | Loss:0.09696717402623402 | top1:96.26394653320312
251/502 Data:0.001 | Batch:0.330 | Total:0:01:26 | ETA:0:01:28 | Loss:0.0970366344271428 | top1:96.25872039794922
261/502 Data:0.001 | Batch:0.349 | Total:0:01:29 | ETA:0:01:20 | Loss:0.09709604860950703 | top1:96.2673568725586
271/502 Data:0.002 | Batch:0.319 | Total:0:01:32 | ETA:0:01:20 | Loss:0.097062513012

391/502 Data:0.000 | Batch:0.317 | Total:0:02:13 | ETA:0:00:40 | Loss:0.09745963019750002 | top1:96.28556823730469
401/502 Data:0.001 | Batch:0.352 | Total:0:02:16 | ETA:0:00:35 | Loss:0.09757602513022257 | top1:96.27688598632812
411/502 Data:0.001 | Batch:0.340 | Total:0:02:20 | ETA:0:00:32 | Loss:0.09740844747808437 | top1:96.28668212890625
421/502 Data:0.001 | Batch:0.320 | Total:0:02:23 | ETA:0:00:28 | Loss:0.09721831711907285 | top1:96.29231262207031
431/502 Data:0.003 | Batch:0.334 | Total:0:02:26 | ETA:0:00:24 | Loss:0.09753139149354395 | top1:96.28135681152344
441/502 Data:0.001 | Batch:0.331 | Total:0:02:30 | ETA:0:00:23 | Loss:0.097685599678498 | top1:96.28064727783203
451/502 Data:0.001 | Batch:0.358 | Total:0:02:34 | ETA:0:00:19 | Loss:0.09753439894272324 | top1:96.28862762451172
461/502 Data:0.001 | Batch:0.335 | Total:0:02:37 | ETA:0:00:14 | Loss:0.09747531366839585 | top1:96.28948974609375
471/502 Data:0.001 | Batch:0.353 | Total:0:02:40 | ETA:0:00:11 | Loss:0.0976809130

71/502 Data:0.001 | Batch:0.318 | Total:0:00:24 | ETA:0:02:22 | Loss:0.09907849668197229 | top1:96.14876556396484
81/502 Data:0.001 | Batch:0.345 | Total:0:00:28 | ETA:0:02:19 | Loss:0.10000147619917069 | top1:96.12751007080078
91/502 Data:0.001 | Batch:0.328 | Total:0:00:31 | ETA:0:02:19 | Loss:0.10029370759378423 | top1:96.1195068359375
101/502 Data:0.001 | Batch:0.431 | Total:0:00:35 | ETA:0:02:16 | Loss:0.09931549739719618 | top1:96.19817352294922
111/502 Data:0.001 | Batch:0.347 | Total:0:00:38 | ETA:0:02:20 | Loss:0.09933637401407903 | top1:96.19229125976562
121/502 Data:0.001 | Batch:0.318 | Total:0:00:41 | ETA:0:02:05 | Loss:0.09996134095078657 | top1:96.20673370361328
131/502 Data:0.001 | Batch:0.320 | Total:0:00:45 | ETA:0:02:04 | Loss:0.09889105971874172 | top1:96.2458267211914
141/502 Data:0.001 | Batch:0.340 | Total:0:00:48 | ETA:0:02:02 | Loss:0.09868674833618157 | top1:96.22395324707031
151/502 Data:0.001 | Batch:0.340 | Total:0:00:51 | ETA:0:01:57 | Loss:0.0984751513314

271/502 Data:0.001 | Batch:0.317 | Total:0:01:32 | ETA:0:01:15 | Loss:0.09679601521564586 | top1:96.25807189941406
281/502 Data:0.001 | Batch:0.324 | Total:0:01:35 | ETA:0:01:13 | Loss:0.09661723308768985 | top1:96.26751708984375
291/502 Data:0.002 | Batch:0.328 | Total:0:01:38 | ETA:0:01:10 | Loss:0.09680240780524782 | top1:96.27094268798828
301/502 Data:0.001 | Batch:0.371 | Total:0:01:42 | ETA:0:01:06 | Loss:0.09716855001905035 | top1:96.25726318359375
311/502 Data:0.001 | Batch:0.318 | Total:0:01:45 | ETA:0:01:07 | Loss:0.09684713124941399 | top1:96.27462005615234
321/502 Data:0.001 | Batch:0.347 | Total:0:01:48 | ETA:0:01:02 | Loss:0.09664673579862557 | top1:96.2799301147461
331/502 Data:0.001 | Batch:0.338 | Total:0:01:52 | ETA:0:00:58 | Loss:0.09654217221270515 | top1:96.27313232421875
341/502 Data:0.001 | Batch:0.344 | Total:0:01:55 | ETA:0:00:55 | Loss:0.09652198444713246 | top1:96.27130889892578
351/502 Data:0.001 | Batch:0.367 | Total:0:01:58 | ETA:0:00:50 | Loss:0.096754284

471/502 Data:0.001 | Batch:0.324 | Total:0:02:39 | ETA:0:00:11 | Loss:0.0965814993038552 | top1:96.27289581298828
481/502 Data:0.001 | Batch:0.318 | Total:0:02:42 | ETA:0:00:08 | Loss:0.0966235784631757 | top1:96.26510620117188
491/502 Data:0.001 | Batch:0.326 | Total:0:02:46 | ETA:0:00:04 | Loss:0.09637807063803897 | top1:96.27116394042969
501/502 Data:0.001 | Batch:0.322 | Total:0:02:49 | ETA:0:00:01 | Loss:0.0963314900736133 | top1:96.26605987548828
161/161 Data:0.000 | Batch:0.048 | Total:0:00:15 | ETA:0:00:00 | Loss:0.03590976817074966 | top1:98.7975082397461
